<a href="https://colab.research.google.com/github/stevenbowler/EmployeeSurvey/blob/main/UploadFoldersToGrokCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dependancies
import os
from xai_sdk import Client  # Or use openai: from openai import OpenAI
from google.colab import userdata

In [ ]:
# Config
API_KEY = userdata.get('XAI_API_KEY')  # From console.x.ai
COLLECTION_ID = userdata.get('GROK_EMPLOYEE_SURVEY_COLLECTION')  # From collection details
FOLDER_PATH = userdata.get('PDF_FOLDER_PATH')  # Local path to your root folder

In [ ]:
# Initialize client (xAI SDK)
client = Client(api_key=API_KEY)

# Function to upload a single file with metadata
def upload_file(file_path, collection_id):
    with open(file_path, "rb") as f:
        # Upload the file (gets a File ID)
        file = client.files.create(
            file=f,
            purpose="assistants"  # For collections/RAG
        )

    # Add to collection with metadata for structure
    client.collections.add_file(
        collection_id=collection_id,
        file_id=file.id,
        metadata={
            "original_path": file_path.replace(FOLDER_PATH, "").lstrip("/"),  # e.g., "subfolder1/document.pdf"
            "file_type": os.path.splitext(file_path)[1]
        }
    )
    print(f"Uploaded: {file_path} (File ID: {file.id})")

# Walk through folder structure
for root, dirs, files in os.walk(FOLDER_PATH):
    for file in files:
        file_path = os.path.join(root, file)
        if file_path.lower().endswith(('.pdf', '.txt', '.md', '.docx')):  # Filter supported types
            upload_file(file_path, COLLECTION_ID)

print("Upload complete! Check your collection in the console.")